# Modeling NHL Game Outcomes

Building binary classification models to predict the winning team

# Overview

Topline goal: predict the winner of a given NHL game

The first purpose of this model is to see if the winner of an NHL game can be predicted with more accuracy than with a naive prediction - choosing the home team to win every time with no other informative input.

In this notebook several modeling techniques will be trained/tested using team game log data from the prior three full NHL regular seasons (19'-20', 20'-21', 21'-22). Games played to date from the current season (22'-23') will then be used to evaluate the model's ability to correctly predict the winning team.

If I can develop a game prediction model that performs above that naive baseline, the next step will be to adjust the benchmark to a semi naive prediction. The semi naive baseline will be based on choosing the 'Vegas' favorite to win everytime. The recent wave of mobile sports betting legalization has led to massive increase in sports wagers placed both in the US and globally. With the increased popularity of sports betting, sportsbooks odds are an increasingly efficient market i.e. the implied probabilties of sportsbook odds are an unbiased estimator of outcomes.

If I can develop a model which is competitive with the betting market, the next step will be to test betting strategies which leverage the predition model to yield postive ROI. Given the 'vig' (i.e. commission) charged by sportsbooks to take wagers, simply beating the market will not be enough to produce a profitable strategy. Instead the model will have to outperform the market by ~5+% to approach profitability.

The excess return required to be profitable in sports betting and generally high log loss scores in betting models, necessitates the application of betting strategies to the model's prediction. Rather than betting equal amounts on each event, I will test strategies that aim to identify and capitilize on over/undervalued sportsbooks odds.

In [15]:
# Standard Packages
import pandas as pd
from pandas.testing import assert_frame_equal
import numpy as np
import re
import time
import os
import warnings
import pickle

# Viz Packages
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

# Scraping packages
import requests
import json
from bs4 import BeautifulSoup
import hockey_scraper

# Modeling Packages
## Modeling Prep
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold, \
GridSearchCV, RandomizedSearchCV

## SKLearn Data Prep Modules
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, \
PolynomialFeatures, PowerTransformer, Normalizer, MaxAbsScaler
from sklearn.impute import SimpleImputer

## SKLearn Classification Models
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,\
ExtraTreesClassifier, VotingClassifier, AdaBoostClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC

## SKLearn Pipeline Setup
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer

## SKLearn Model Optimization
from sklearn.feature_selection import RFE, f_regression, RFECV, SelectKBest

# ## Boosting
# from xgboost import XGBRegressor
from xgboost import XGBClassifier

## SKLearn Metrics
### Classification Scoring/Evaluation
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score, \
ConfusionMatrixDisplay, log_loss, confusion_matrix, RocCurveDisplay, make_scorer, roc_auc_score

In [5]:
# Notebook Config
from pprintpp import pprint as pp
%reload_ext pprintpp
from tqdm import tqdm
from io import StringIO

## Suppress Python Warnings (Future, Deprecation)
warnings.filterwarnings("ignore", category= FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Suppress Pandas Warnings (SettingWithCopy)
pd.options.mode.chained_assignment = None

## Pandas Display Config
pd.options.display.max_columns = 80
# pd.options.display.width = None

## Display SKLearn estimators as diagrams
from sklearn import set_config
set_config(display= 'diagram')

## EDA

### Data Retrieval

Data notes:
- Includes offensive, defensive and goaltending stats for both the home and away teams
- For each team data for 3 different game strength states is used
    - 5v5 even strength (adjusted for score states and venue)
    - 5v4 powerplay (man advantage)
    - 4v5 penalty pill (man down)
- Static data from prior season is used for initial development
- Once the model is built, I will aim to transition to a dynamic/automated ETL process

In [40]:
# Load test/train data 
# Static game log data from prior 3 full seasons
# Team stats for both sides in even strength (adjusted), man up and man down situations

home_5v5_adj = pd.read_csv('data/filtered/filtered-19_22-home-5v5_adj.csv')
home_5v4_pp = pd.read_csv('data/filtered/filtered-19_22-home-pp_5v4.csv')
home_4v5_pk = pd.read_csv('data/filtered/filtered-19_22-home-pk_4v5.csv')

away_5v5_adj = pd.read_csv('data/filtered/filtered-19_22-away-5v5_adj.csv')
away_5v4_pp = pd.read_csv('data/filtered/filtered-19_22-away-pp_5v4.csv')
away_4v5_pk = pd.read_csv('data/filtered/filtered-19_22-away-pk_4v5.csv')

print(home_5v5_adj.shape, home_5v4_pp.shape, home_4v5_pk.shape)
print(away_5v5_adj.shape, away_5v4_pp.shape, away_4v5_pk.shape)

(3262, 33) (3205, 33) (3191, 33)
(3262, 33) (3191, 33) (3205, 33)


In [29]:
home_5v5_adj.head()

,Game,Team,Unnamed: 2,TOI,CF/60,CA/60,CF%,FF/60,FA/60,FF%,...,HDCA/60,HDCF%,HDGF/60,HDGA/60,HDGF%,HDSH%,HDSV%,SH%,SV%,PDO
0,"2019-10-02 - Senators 3, Maple Leafs 5",Toronto Maple Leafs,Limited ReportFull Report,44.133333,78.00,54.02,59.08,59.15,43.75,57.48,...,8.32,65.42,5.12,4.20,54.92,55.99,24.79,11.94,85.60,0.975
1,"2019-10-02 - Capitals 3, Blues 2",St Louis Blues,Limited ReportFull Report,50.866667,33.43,47.21,41.45,27.17,39.14,40.97,...,6.14,47.66,1.11,0.00,100.00,19.64,100.00,5.56,95.79,1.014
2,"2019-10-02 - Canucks 2, Oilers 3",Edmonton Oilers,Limited ReportFull Report,47.066667,44.76,67.92,39.72,30.38,48.03,38.75,...,9.11,39.87,2.40,1.35,64.02,39.29,65.85,14.87,91.33,1.062
3,"2019-10-02 - Sharks 1, Golden Knights 4",Vegas Golden Knights,Limited ReportFull Report,45.666667,62.73,40.18,60.96,51.94,30.07,63.34,...,5.27,75.25,1.30,0.00,100.00,12.06,100.00,8.07,93.28,1.013
4,"2019-10-03 - Panthers 2, Lightning 5",Tampa Bay Lightning,Limited ReportFull Report,45.500000,46.81,57.60,44.83,32.45,46.56,41.07,...,9.26,55.98,1.31,1.33,49.49,19.81,83.02,9.94,96.45,1.064


In [30]:
all_cols = (home_5v5_adj.columns.to_list())
pp(all_cols)

[
    'Game',
    'Team',
    'Unnamed: 2',
    'TOI',
    'CF/60',
    'CA/60',
    'CF%',
    'FF/60',
    'FA/60',
    'FF%',
    'SF/60',
    'SA/60',
    'SF%',
    'GF/60',
    'GA/60',
    'GF%',
    'xGF/60',
    'xGA/60',
    'xGF%',
    'SCF/60',
    'SCA/60',
    'SCF%',
    'HDCF/60',
    'HDCA/60',
    'HDCF%',
    'HDGF/60',
    'HDGA/60',
    'HDGF%',
    'HDSH%',
    'HDSV%',
    'SH%',
    'SV%',
    'PDO',
]


In [31]:
# Even strength features list
# comment out features to disregard
ev_features_list = [
    'Game',
    'Team',
    'TOI',
#     'CF/60',
#     'CA/60',
#     'CF%',
    # 'FF/60',
    # 'FA/60',
    'FF%',
#     'SF/60',
#     'SA/60',
#     'SF%',
    # 'GF/60',
    # 'GA/60',
    'GF%',
    # 'xGF/60',
    # 'xGA/60',
    'xGF%',
#     'SCF/60',
#     'SCA/60',
#     'SCF%',
    # 'HDCF/60',
    # 'HDCA/60',
    'HDCF%',
#     'HDGF/60',
#     'HDGA/60',
#     'HDGF%',
    'HDSH%',
    'HDSV%',
    'SH%',
    'SV%',
#    'PDO'
]

In [32]:
pp_cols = (home_5v4_pp.columns.to_list())
print(pp_cols)

['Game', 'Team', 'Unnamed: 2', 'TOI', 'CF/60', 'CA/60', 'CF%', 'FF/60', 'FA/60', 'FF%', 'SF/60', 'SA/60', 'SF%', 'GF/60', 'GA/60', 'GF%', 'xGF/60', 'xGA/60', 'xGF%', 'SCF/60', 'SCA/60', 'SCF%', 'HDCF/60', 'HDCA/60', 'HDCF%', 'HDGF/60', 'HDGA/60', 'HDGF%', 'HDSH%', 'HDSV%', 'SH%', 'SV%', 'PDO']


In [33]:
# PP df features list
# comment out features to disregard
pp_features_list = [
#     'Game',
#     'Team',
    'TOI',
#     'CF/60',
#     'CA/60',
#     'CF%',
#     'FF/60',
#     'FA/60',
#     'FF%',
#     'SF/60',
#     'SA/60',
#     'SF%',
    'GF/60',
#     'GA/60',
#     'GF%',
    'xGF/60',
#     'xGA/60',
#     'xGF%',
#     'SCF/60',
#     'SCA/60',
#     'SCF%',
#     'HDCF/60',
#     'HDCA/60',
#     'HDCF%',
#     'HDGF/60',
#     'HDGA/60',
#     'HDGF%',
#     'HDSH%',
#     'HDSV%',
#     'SH%',
#     'SV%',
#     'PDO'
]

In [34]:
# PK df features list
# same as pp df but swap GFs for GAs
pk_features_list = ['TOI', 'GA/60', 'xGF/60']

In [99]:
# team abbreviation dict from hockey scraper package function
# "_descrition": "# All the corresponding tri-codes for team names",

team_dict = {
    'Anaheim Ducks': 'ANA',
    'Arizona Coyotes' : 'ARI',
    'Boston Bruins': 'BOS', 
    'Buffalo Sabres':'BUF',
    'Calgary Flames': 'CGY', 
    'Carolina Hurricanes': 'CAR', 
    'Chicago Blackhawks': 'CHI', 
    'Colorado Avalanche': 'COL',
    'Columbus Blue Jackets': 'CBJ',
    'Dallas Stars': 'DAL',
    'Detroit Red Wings': 'DET',
    'Edmonton Oilers': 'EDM',
    'Florida Panthers': 'FLA',
    'Los Angeles Kings': 'L.A.',
    'Minnesota Wild': 'MIN',
    'Montreal Canadiens': 'MTL',
    'Nashville Predators': 'NSH',
    'New Jersey Devils': 'N.J.',
    "New York Islanders": 'NYI',
    "New York Rangers": 'NYR',
    'Ottawa Senators': 'OTT',
    'Philadelphia Flyers': 'PHI',
    'Pittsburgh Penguins': 'PIT',
    'San Jose Sharks': 'S.J.',
    'Seattle Kraken': 'SEA',
#     'St. Louis Blues': 'STL',
    'St Louis Blues': 'STL',
    'Tampa Bay Lightning': 'T.B.',
    'Toronto Maple Leafs': 'TOR',
    'Vancouver Canucks': 'VAN',
    'Vegas Golden Knights':'VGK',
    'Washington Capitals': 'WSH',
    'Winnipeg Jets': 'WPG'
}

In [100]:
len(teams)

32

In [112]:
# 1st round cleaning of data sets
def process_datasets(game_log_df):
    # extract date from game column
    game_log_df['Date'] = pd.to_datetime(game_log_df['Game'].str[:11])
    # sub full team names with abbreviated names
#     game_log_df = game_log_df.replace({'Team': team_dict})
    game_log_df.replace(team_dict, inplace=True)
    # create game log index
    game_log_df['Game_Key'] = game_log_df['Team'].astype(str)+'_'+game_log_df['Date'].astype(str)
    return game_log_df

In [113]:
process_datasets(home_5v5_adj)
home_5v5_adj.head()

,Game,Team,Unnamed: 2,TOI,CF/60,CA/60,CF%,FF/60,FA/60,FF%,...,HDGF/60,HDGA/60,HDGF%,HDSH%,HDSV%,SH%,SV%,PDO,Date,Game_Key
0,"2019-10-02 - Senators 3, Maple Leafs 5",TOR,Limited ReportFull Report,44.133333,78.00,54.02,59.08,59.15,43.75,57.48,...,5.12,4.20,54.92,55.99,24.79,11.94,85.60,0.975,2019-10-02,TOR_2019-10-02
1,"2019-10-02 - Capitals 3, Blues 2",STL,Limited ReportFull Report,50.866667,33.43,47.21,41.45,27.17,39.14,40.97,...,1.11,0.00,100.00,19.64,100.00,5.56,95.79,1.014,2019-10-02,STL_2019-10-02
2,"2019-10-02 - Canucks 2, Oilers 3",EDM,Limited ReportFull Report,47.066667,44.76,67.92,39.72,30.38,48.03,38.75,...,2.40,1.35,64.02,39.29,65.85,14.87,91.33,1.062,2019-10-02,EDM_2019-10-02
3,"2019-10-02 - Sharks 1, Golden Knights 4",VGK,Limited ReportFull Report,45.666667,62.73,40.18,60.96,51.94,30.07,63.34,...,1.30,0.00,100.00,12.06,100.00,8.07,93.28,1.013,2019-10-02,VGK_2019-10-02
4,"2019-10-03 - Panthers 2, Lightning 5",T.B.,Limited ReportFull Report,45.500000,46.81,57.60,44.83,32.45,46.56,41.07,...,1.31,1.33,49.49,19.81,83.02,9.94,96.45,1.064,2019-10-03,T.B._2019-10-03


In [114]:
home_5v5_adj['Team'].value_counts()

WPG     106
DET     106
OTT     106
VGK     106
MTL     106
CBJ     105
ANA     105
S.J.    105
NYR     105
NSH     104
PIT     104
NYI     104
BOS     104
PHI     104
STL     104
BUF     104
MIN     104
FLA     104
VAN     104
CHI     103
L.A.    103
TOR     103
N.J.    103
DAL     103
T.B.    103
EDM     103
CGY     102
ARI     102
WSH     102
COL     102
CAR     102
SEA      41
Name: Team, dtype: int64

In [131]:
process_datasets(home_5v4_pp)
process_datasets(home_4v5_pk)
process_datasets(away_5v5_adj)
process_datasets(away_5v4_pp)
process_datasets(away_4v5_pk)

,Game,Team,Unnamed: 2,TOI,CF/60,CA/60,CF%,FF/60,FA/60,FF%,...,HDGF/60,HDGA/60,HDGF%,HDSH%,HDSV%,SH%,SV%,PDO,Date,Game_Key
0,"2019-10-02 - Senators 3, Maple Leafs 5",OTT,Limited ReportFull Report,8.166667,14.69,146.94,9.09,14.69,124.9,10.53,...,0,0,-,0.00,100.00,0.00,90.00,0.900,2019-10-02,OTT_2019-10-02
1,"2019-10-02 - Capitals 3, Blues 2",WSH,Limited ReportFull Report,3.250000,18.46,92.31,16.67,18.46,73.85,20.00,...,0,0,-,-,-,0.00,50.00,0.500,2019-10-02,WSH_2019-10-02
2,"2019-10-02 - Canucks 2, Oilers 3",VAN,Limited ReportFull Report,4.000000,0,120,0.00,0,45,0.00,...,0,0,-,-,100.00,-,100.00,-,2019-10-02,VAN_2019-10-02
3,"2019-10-02 - Sharks 1, Golden Knights 4",S.J.,Limited ReportFull Report,4.333333,0,152.31,0.00,0,124.62,0.00,...,0,0,-,-,-,-,83.33,-,2019-10-02,S.J._2019-10-02
4,"2019-10-03 - Panthers 2, Lightning 5",FLA,Limited ReportFull Report,4.233333,14.17,99.21,12.50,14.17,99.21,12.50,...,0,14.17,0.00,-,50.00,100.00,85.71,1.857,2019-10-03,FLA_2019-10-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3200,"2022-04-29 - Avalanche 1, Wild 4",COL,Limited ReportFull Report,9.983333,0,84.14,0.00,0,54.09,0.00,...,0,0,-,-,-,-,100.00,-,2022-04-29,COL_2022-04-29
3201,"2022-04-29 - Flames 1, Jets 3",CGY,Limited ReportFull Report,5.400000,11.11,100,10.00,11.11,77.78,12.50,...,0,0,-,-,100.00,0.00,100.00,1.000,2022-04-29,CGY_2022-04-29
3202,"2022-04-29 - Predators 4, Coyotes 5",NSH,Limited ReportFull Report,5.950000,10.08,110.92,8.33,0,80.67,0.00,...,0,0,-,-,100.00,-,100.00,-,2022-04-29,NSH_2022-04-29
3203,"2022-04-29 - Sharks 0, Kraken 3",S.J.,Limited ReportFull Report,2.000000,0,150,0.00,0,90,0.00,...,0,0,-,-,100.00,-,100.00,-,2022-04-29,S.J._2022-04-29


In [137]:
home_5v4_pp['TOI']

0       8.166667
1       3.250000
2       4.000000
3       4.333333
4       4.233333
          ...   
3200    9.983333
3201    5.400000
3202    5.950000
3203    2.000000
3204    2.000000
Name: TOI, Length: 3205, dtype: float64

In [138]:
home_5v4_pp['xGF/60']

0        7.47
1        6.44
2         4.6
3        8.96
4       10.54
        ...  
3200      3.6
3201     9.52
3202    15.61
3203    12.69
3204     5.95
Name: xGF/60, Length: 3205, dtype: object

In [142]:
home_5v4_pp['GF/60']

0        7.35
1       18.46
2           0
3       13.85
4       14.17
        ...  
3200        0
3201        0
3202        0
3203        0
3204        0
Name: GF/60, Length: 3205, dtype: object

### Add NHL.com schedule/result df


In [ ]:
schedule_df['Home_Team_Won'] = np.where(sched_df['home_score'] > sched_df['away_score'], 1, 0)
schedule_df['Home_Team_Key'] = schedule_df['home_team'].astype(str)+'_'+sched_df['date'].astype(str)
schedule_df['Away_Team_Key'] = schedule_df['away_team'].astype(str)+'_'+sched_df['date'].astype(str)

### Preliminary Feauture Selection and Processing

In [133]:
# add xg per pp minute

### Top line feature (column) deletion and addition

Deleting:
- 'Unnamed: 2' held hyperlinks to related pages
- 'C' type (Corsi) shot stats 
  - we will be using 'F' type (Fenwick) shots (goals, on net and misses, but excludes blocks)
- 'SC' type (Scoring Chance) vars
    - we will be focusing on HD type